Feature extraction (run next block first)

In [3]:
from sklearn.linear_model import LinearRegression
import numpy as np
import json
import pandas as pd

def extractValuefromSpectrum(startIndx,endIndx,spectrum):
    return [spectrum[str(indx)] for indx in range(startIndx,endIndx+1)]

Low_cali_linear_loss = [-21.144444444444446, -21.200000000000003, -21.26666666666667, -21.155555555555555, -21.13333333333333, -12.5, -12.722222222222225, -12.666666666666666, -12.6, -12.733333333333336, -12.722222222222225, -12.688888888888892, -12.688888888888892, -12.7, -12.722222222222221, -12.7, -12.68888888888889, -12.622222222222224, -12.600000000000001, -12.68888888888889, -12.666666666666666, -12.766666666666667, -12.733333333333334, -12.68888888888889, -12.700000000000001, -12.7, -12.655555555555557, -12.711111111111114, -12.68888888888889, -12.677777777777782, -12.58888888888889, -12.566666666666666, -12.5, -12.566666666666666, -12.677777777777779, -12.76666666666667, -12.677777777777777, -12.68888888888889, -12.644444444444446, -12.67777777777778, -12.722222222222225, -12.633333333333335, -12.644444444444446, -12.700000000000001, -12.81111111111111, -12.799999999999999, -12.755555555555558, -12.711111111111114, -12.722222222222221, -12.688888888888888, -12.711111111111114, -12.700000000000001, -12.788888888888891, -12.76666666666667, -12.68888888888889, -12.622222222222224, -12.700000000000001, -12.777777777777777, -12.744444444444447, -12.633333333333333, -12.655555555555557, -12.58888888888889, -12.622222222222224, -12.633333333333335, -12.633333333333335, -12.655555555555557, -12.733333333333334, -12.755555555555553, -12.68888888888889, -12.71111111111111, -12.777777777777779, -12.855555555555554, -12.8, -12.722222222222221, -12.844444444444443, -12.75555555555556, -12.899999999999999, -12.855555555555554, -12.877777777777778, -12.811111111111112, -12.81111111111111, -12.800000000000004, -12.799999999999999, -12.788888888888888, -12.777777777777775, -12.799999999999999, -12.844444444444441, -12.855555555555554, -12.866666666666667, -12.788888888888886, -12.844444444444443, -12.866666666666667, -12.97777777777778, -12.955555555555556, -12.922222222222222]
#this function only report the input/output spectra per experiment data chunk
def before_gain_calculation(buffer):
    input_spectras = []
    output_spectras= []
    input_spectras = buffer["roadm_dut_line_out_spectra"]
    input_spectras = [input_spectras[str(i+1)] for i in range(len(input_spectras))]
    output_spectras = buffer["roadm_aux_line_in_spectra"]
    output_spectras = [output_spectras[str(i+1)] for i in range(len(output_spectras))]
    

    input_spectras = np.asarray(input_spectras)
    output_spectras = np.asarray(output_spectras)
    return input_spectras,output_spectras


def featureExtraction(data_characterization):
    ExtractedFeature = pd.DataFrame()
    for metadata in data_characterization:
        channelNum = metadata["num_opened_channels"]
        wss_channels = metadata["open_channels"]
        fiber_in_power = metadata["roadm_dut_line_port"]["output-power"]
        fiber_out_power = metadata["roadm_aux_line_port"]["input-power"]

        fibre_in_spectrum = metadata["roadm_dut_line_out_spectra"]
        fibre_in = extractValuefromSpectrum(6,95,fibre_in_spectrum)
        fibre_out_spectrum  = metadata["roadm_aux_line_in_spectra"]
        fibre_out = extractValuefromSpectrum(6,95,fibre_out_spectrum)
        
        DUT_WSS_activated_channels = [0]*90
        # print(wss_channels)
        for indx in wss_channels:
            # print(indx)
            DUT_WSS_activated_channels[indx-1-5] = 1
        # write the PD power info
        metaResult = {}
        metaResult['linear_loss'] = +2.54 # ?
        metaResult['Total_fiber_in_power'] = fiber_in_power
        metaResult['Total_fiber_out_power'] = fiber_out_power
        # write the spectra info
        for idx in range(90):
            i = 90-idx-1 # flip the indx to be consist with ireland side.
            post_indx = str(idx).zfill(2)
            metaResult['Fibre_input_spectra_'+post_indx] = fibre_in[i]
            metaResult['Activated_channel_index_'+post_indx] = DUT_WSS_activated_channels[i]

        cur_open_channel = np.asarray(metadata["open_channels"]).astype(int) - 1
        cur_total_out_power = metadata["roadm_dut_line_port"]["output-power"]
        cur_total_in_power = metadata["roadm_aux_line_port"]["input-power"]
        cur_input, cur_output = before_gain_calculation(metadata)

        cur_output_interest = [cur_output[i] for i in cur_open_channel]
        cur_linear_loss_interest = [Low_cali_linear_loss[i] for i in cur_open_channel]
        cur_input_interest = [cur_input[i] for i in cur_open_channel]
        cur_output_interest = np.array(cur_output_interest) - np.array(cur_linear_loss_interest)

        cur_gain_interest = cur_output_interest# - cur_input_interest

        cur_coefficients = np.polyfit(cur_open_channel, cur_gain_interest, 2)
        cur_polynomial = np.poly1d(cur_coefficients)
        cur_fit_gain = cur_polynomial(cur_open_channel)

        cur_raman_tilt = cur_fit_gain[0] - cur_fit_gain[-1]
        metaResult['Raman_tilt'] = cur_raman_tilt
        # break
        
        # ExtractedFeature = ExtractedFeature.append([metaResult],ignore_index=True)
        ExtractedFeature = pd.concat([ExtractedFeature,pd.DataFrame.from_dict([metaResult])],ignore_index=True)

    return ExtractedFeature

def featureExtraction_random(data_characterization,channel_number):
    ExtractedFeature = pd.DataFrame()
    for metadata in data_characterization:
        channelNum = metadata["num_opened_channels"]
        wss_channels = metadata["open_channels"]
        fiber_in_power = metadata["roadm_dut_line_port"]["output-power"]
        fiber_out_power = metadata["roadm_aux_line_port"]["input-power"]
        if channelNum != channel_number: continue

        fibre_in_spectrum = metadata["roadm_dut_line_out_spectra"]
        fibre_in = extractValuefromSpectrum(6,95,fibre_in_spectrum)
        fibre_out_spectrum  = metadata["roadm_aux_line_in_spectra"]
        fibre_out = extractValuefromSpectrum(6,95,fibre_out_spectrum)
        
        DUT_WSS_activated_channels = [0]*90
        # print(wss_channels)
        for indx in wss_channels:
            # print(indx)
            DUT_WSS_activated_channels[indx-1-5] = 1
        # write the PD power info
        metaResult = {}
        metaResult['linear_loss'] = +2.54 # ?
        metaResult['Total_fiber_in_power'] = fiber_in_power
        metaResult['Total_fiber_out_power'] = fiber_out_power
        # write the spectra info
        for idx in range(90):
            i = 90-idx-1 # flip the indx to be consist with ireland side.
            post_indx = str(idx).zfill(2)
            metaResult['Fibre_input_spectra_'+post_indx] = fibre_in[i]
            metaResult['Activated_channel_index_'+post_indx] = DUT_WSS_activated_channels[i]

        cur_open_channel = np.asarray(metadata["open_channels"]).astype(int) - 1
        cur_total_out_power = metadata["roadm_dut_line_port"]["output-power"]
        cur_total_in_power = metadata["roadm_aux_line_port"]["input-power"]
        cur_input, cur_output = before_gain_calculation(metadata)

        cur_output_interest = [cur_output[i] for i in cur_open_channel]
        cur_linear_loss_interest = [Low_cali_linear_loss[i] for i in cur_open_channel]
        cur_input_interest = [cur_input[i] for i in cur_open_channel]
        cur_output_interest = np.array(cur_output_interest) - np.array(cur_linear_loss_interest)

        cur_gain_interest = cur_output_interest# - cur_input_interest

        cur_coefficients = np.polyfit(cur_open_channel, cur_gain_interest, 2)
        cur_polynomial = np.poly1d(cur_coefficients)
        cur_fit_gain = cur_polynomial(cur_open_channel)

        cur_raman_tilt = cur_fit_gain[0] - cur_fit_gain[-1]
        metaResult['Raman_tilt'] = cur_raman_tilt
        # break
        
        # ExtractedFeature = ExtractedFeature.append([metaResult],ignore_index=True)
        ExtractedFeature = pd.concat([ExtractedFeature,pd.DataFrame.from_dict([metaResult])],ignore_index=True)

    return ExtractedFeature

In [4]:
train_ratio = 0.8
data_prepath = "../data/"
output_prepath = "../feature2/"

fileList = ['fibre_25km_step', 'fibre_25km_random', 'fibre_25km_edge_goalpost', 
            'fibre_50km_step', 'fibre_50km_random', 'fibre_50km_edge_goalpost']

# Initialize DataFrames for different combinations
extractedFeatures = {}
combinedFeatures = {'25km': pd.DataFrame(), '50km': pd.DataFrame()}

for fileName in fileList:
    print(fileName)
    filePath = data_prepath + fileName + ".json"

    with open(filePath, "r") as read_file:
        data = json.load(read_file)

        # Determine type and distance for filename
        distance = '25km' if '25km' in fileName else '50km'

        # Apply feature extraction based on file type
        if 'step' in fileName:
            fileType = 'step'
            newFeature = featureExtraction(data["measurement_data"])
        elif 'goalpost' in fileName:
            fileType = 'goalpost'
            newFeature = featureExtraction(data["measurement_data"])
        elif 'random' in fileName:
            fileType = 'random'
            newFeature = pd.DataFrame()
            for channel_number in [2, 5, 10, 20, 30, 40, 50, 60, 70, 80, 90]:
                tempFeature = featureExtraction_random(data["measurement_data"], channel_number)
                newFeature = pd.concat([newFeature, tempFeature], ignore_index=True)

        # Key to identify each unique combination of type and distance
        key = f"{distance}_{fileType}"
        if key not in extractedFeatures:
            extractedFeatures[key] = newFeature
        else:
            extractedFeatures[key] = pd.concat([extractedFeatures[key], newFeature], ignore_index=True)

        # Add to combined dataset
        combinedFeatures[distance] = pd.concat([combinedFeatures[distance], newFeature], ignore_index=True)

# Split and output the feature dataframes to CSV files, if needed
for key, df in extractedFeatures.items():
    if 'random' in key:
        training_data = df.sample(frac=train_ratio, random_state=50)
        testing_data = df.drop(training_data.index)
        training_data.to_csv(output_prepath + key + '183' + "_train.csv", index=False)
        testing_data.to_csv(output_prepath + key + '183' + "_test.csv", index=False)
    else:
        df.to_csv(output_prepath + key + '183' + ".csv", index=False)

# Split and output the combined dataframes
for distance, df in combinedFeatures.items():
    training_data = df.sample(frac=train_ratio, random_state=50)
    testing_data = df.drop(training_data.index)
    training_data.to_csv(output_prepath + distance + '_combined' + '183' + "_train.csv", index=False)
    testing_data.to_csv(output_prepath + distance + '_combined' + '183' + "_test.csv", index=False)

fibre_25km_step
fibre_25km_random


/var/folders/5d/_vt9x59177jdykxfrjyb3q_r0000gn/T/ipykernel_3776/2212543836.py:67: RankWarning: Polyfit may be poorly conditioned
  cur_coefficients = np.polyfit(cur_open_channel, cur_gain_interest, 2)
/var/folders/5d/_vt9x59177jdykxfrjyb3q_r0000gn/T/ipykernel_3776/2212543836.py:67: RankWarning: Polyfit may be poorly conditioned
  cur_coefficients = np.polyfit(cur_open_channel, cur_gain_interest, 2)
/var/folders/5d/_vt9x59177jdykxfrjyb3q_r0000gn/T/ipykernel_3776/2212543836.py:123: RankWarning: Polyfit may be poorly conditioned
  cur_coefficients = np.polyfit(cur_open_channel, cur_gain_interest, 2)
/var/folders/5d/_vt9x59177jdykxfrjyb3q_r0000gn/T/ipykernel_3776/2212543836.py:123: RankWarning: Polyfit may be poorly conditioned
  cur_coefficients = np.polyfit(cur_open_channel, cur_gain_interest, 2)
/var/folders/5d/_vt9x59177jdykxfrjyb3q_r0000gn/T/ipykernel_3776/2212543836.py:123: RankWarning: Polyfit may be poorly conditioned
  cur_coefficients = np.polyfit(cur_open_channel, cur_gain_inter

fibre_25km_edge_goalpost


/var/folders/5d/_vt9x59177jdykxfrjyb3q_r0000gn/T/ipykernel_3776/2212543836.py:67: RankWarning: Polyfit may be poorly conditioned
  cur_coefficients = np.polyfit(cur_open_channel, cur_gain_interest, 2)
/var/folders/5d/_vt9x59177jdykxfrjyb3q_r0000gn/T/ipykernel_3776/2212543836.py:67: RankWarning: Polyfit may be poorly conditioned
  cur_coefficients = np.polyfit(cur_open_channel, cur_gain_interest, 2)
/var/folders/5d/_vt9x59177jdykxfrjyb3q_r0000gn/T/ipykernel_3776/2212543836.py:67: RankWarning: Polyfit may be poorly conditioned
  cur_coefficients = np.polyfit(cur_open_channel, cur_gain_interest, 2)
/var/folders/5d/_vt9x59177jdykxfrjyb3q_r0000gn/T/ipykernel_3776/2212543836.py:67: RankWarning: Polyfit may be poorly conditioned
  cur_coefficients = np.polyfit(cur_open_channel, cur_gain_interest, 2)
/var/folders/5d/_vt9x59177jdykxfrjyb3q_r0000gn/T/ipykernel_3776/2212543836.py:67: RankWarning: Polyfit may be poorly conditioned
  cur_coefficients = np.polyfit(cur_open_channel, cur_gain_interest

fibre_50km_step
fibre_50km_random


/var/folders/5d/_vt9x59177jdykxfrjyb3q_r0000gn/T/ipykernel_3776/2212543836.py:123: RankWarning: Polyfit may be poorly conditioned
  cur_coefficients = np.polyfit(cur_open_channel, cur_gain_interest, 2)
/var/folders/5d/_vt9x59177jdykxfrjyb3q_r0000gn/T/ipykernel_3776/2212543836.py:123: RankWarning: Polyfit may be poorly conditioned
  cur_coefficients = np.polyfit(cur_open_channel, cur_gain_interest, 2)
/var/folders/5d/_vt9x59177jdykxfrjyb3q_r0000gn/T/ipykernel_3776/2212543836.py:123: RankWarning: Polyfit may be poorly conditioned
  cur_coefficients = np.polyfit(cur_open_channel, cur_gain_interest, 2)
/var/folders/5d/_vt9x59177jdykxfrjyb3q_r0000gn/T/ipykernel_3776/2212543836.py:123: RankWarning: Polyfit may be poorly conditioned
  cur_coefficients = np.polyfit(cur_open_channel, cur_gain_interest, 2)
/var/folders/5d/_vt9x59177jdykxfrjyb3q_r0000gn/T/ipykernel_3776/2212543836.py:123: RankWarning: Polyfit may be poorly conditioned
  cur_coefficients = np.polyfit(cur_open_channel, cur_gain_int

fibre_50km_edge_goalpost


/var/folders/5d/_vt9x59177jdykxfrjyb3q_r0000gn/T/ipykernel_3776/2212543836.py:67: RankWarning: Polyfit may be poorly conditioned
  cur_coefficients = np.polyfit(cur_open_channel, cur_gain_interest, 2)
/var/folders/5d/_vt9x59177jdykxfrjyb3q_r0000gn/T/ipykernel_3776/2212543836.py:67: RankWarning: Polyfit may be poorly conditioned
  cur_coefficients = np.polyfit(cur_open_channel, cur_gain_interest, 2)
/var/folders/5d/_vt9x59177jdykxfrjyb3q_r0000gn/T/ipykernel_3776/2212543836.py:67: RankWarning: Polyfit may be poorly conditioned
  cur_coefficients = np.polyfit(cur_open_channel, cur_gain_interest, 2)
/var/folders/5d/_vt9x59177jdykxfrjyb3q_r0000gn/T/ipykernel_3776/2212543836.py:67: RankWarning: Polyfit may be poorly conditioned
  cur_coefficients = np.polyfit(cur_open_channel, cur_gain_interest, 2)
/var/folders/5d/_vt9x59177jdykxfrjyb3q_r0000gn/T/ipykernel_3776/2212543836.py:67: RankWarning: Polyfit may be poorly conditioned
  cur_coefficients = np.polyfit(cur_open_channel, cur_gain_interest

helper function for feature extraction